In [1]:
import requests
import nltk
import string
import gensim
import pandas as pd
from nltk.corpus import stopwords, wordnet
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
from googletrans import Translator
from bs4 import BeautifulSoup
from gensim import corpora, models

unable to import 'smart_open.gcs', disabling that module


In [2]:
URL = 'https://www.musica.com/letras.asp?letra=1305186'
page = requests.get(URL)
print(page.json)

<bound method Response.json of <Response [200]>>


In [3]:
soup = BeautifulSoup(page.content, 'html.parser')

In [5]:
p_l = []
for p in soup.find_all('p'):
    p_l.append(p)
    
raw_song = list(p_l[1])
filtered_song = []
for j in raw_song:
    if ('p***' in j):
        filtered_song.append(str(j).replace('p***', 'puta'))
        if ('mie***' in j):
            filtered_song.append(str(j).replace('mie***', 'mierda'))
        else: continue
    elif ('mie***' in j):
        filtered_song.append(str(j).replace('mie***', 'mierda'))
        if ('p***' in j):
            filtered_song.append(str(j).replace('p***', 'puta'))
        else: continue
    else:
        filtered_song.append(str(j))  
    
filtered_song = filtered_song[::2]

In [6]:
filtered_song[0:5]

['    Mi padre es el sol, mi madre la luna',
 ' mi hermano es el viento y el planeta tierra mi cuna',
 ' mis unicos hijos son las frases que me invento',
 ' y mi mayor regalo es vivir este momento',
 ' en el que siento que callar es un pecado capital']

In [7]:
translator = Translator()
translate_song = []
for i in translator.translate(filtered_song):
    translate_song.append(i.text)

In [8]:
translate_song[0:5]

['My father is the sun, the moon my mother',
 'my brother is the wind and planet earth my crib',
 'my only children are the phrases that invented me',
 'and my greatest gift is to live this time',
 'in which I feel that silence is a cardinal sin']

In [9]:
word_tokenize = word_tokenize(str(translate_song).lower())

In [10]:
stop_words = stopwords.words('english')
filtered_words = [x for x in word_tokenize if x not in stop_words +list(string.punctuation)+['``',"''"]]

In [11]:
for i, word in enumerate(filtered_words):
    for char in word:
        if char == "'":
            word = word.replace("'",'')
            filtered_words[i] = word
        else:
            filtered_words[i] = word

In [12]:
FreqDist([x for x in filtered_words if not x in stop_words]).most_common(20)

[('rap', 10),
 ('children', 6),
 ('life', 6),
 ('know', 5),
 ('nothing', 5),
 ('see', 5),
 ('go', 5),
 ('want', 4),
 ('another', 4),
 ('look', 4),
 ('like', 4),
 ('also', 4),
 ('give', 4),
 ('although', 4),
 ('love', 4),
 ('time', 3),
 ('feel', 3),
 ('city', 3),
 ('science', 3),
 ('tell', 3)]

In [13]:
translate_song = pd.DataFrame({'Sentences': translate_song})

In [14]:
def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    
    return tag_dict.get(tag, wordnet.NOUN)

In [15]:
def preprocess(translate_song):
    lem = WordNetLemmatizer()
    results = []
    for token in gensim.utils.simple_preprocess(translate_song):
        if token not in list(gensim.parsing.preprocessing.STOPWORDS) +list(string.punctuation):
            results.append(lem.lemmatize(token, get_wordnet_pos(token)))      
    
    return results

In [16]:
preprocess_sentences = translate_song['Sentences'].map(preprocess)

In [17]:
preprocess_sentences[0:5]

0             [father, sun, moon, mother]
1    [brother, wind, planet, earth, crib]
2                 [child, phrase, invent]
3               [great, gift, live, time]
4          [feel, silence, cardinal, sin]
Name: Sentences, dtype: object

In [18]:
dictionary = gensim.corpora.Dictionary(preprocess_sentences)
dictionary.filter_extremes(no_below=3)

corpus = [dictionary.doc2bow(doc) for doc in preprocess_sentences]

In [19]:
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

In [20]:
ida_model = gensim.models.LdaMulticore(corpus_tfidf, num_topics = 3, id2word=dictionary, passes=100)
for idx, topic in ida_model.print_topics(-1): 
    print('Topic: {} \nWords: {}\n'.format(idx, topic))

Topic: 0 
Words: 0.246*"rap" + 0.111*"like" + 0.086*"time" + 0.086*"love" + 0.086*"dream" + 0.073*"fuck" + 0.072*"live" + 0.070*"look" + 0.065*"tell" + 0.009*"child"

Topic: 1 
Words: 0.193*"know" + 0.163*"feel" + 0.129*"play" + 0.118*"come" + 0.110*"kid" + 0.088*"end" + 0.014*"tell" + 0.014*"write" + 0.014*"time" + 0.014*"look"

Topic: 2 
Words: 0.190*"want" + 0.188*"life" + 0.164*"child" + 0.105*"city" + 0.105*"science" + 0.087*"write" + 0.011*"end" + 0.011*"rap" + 0.011*"tell" + 0.011*"look"

